In [2]:
import openpathsampling as p
import openpathsampling.engines.celery.tasks as t
import openpathsampling.netcdfplus as npl
from uuid import UUID
import openpathsampling.engines.openmm as peng
import simtk.openmm as mm
import simtk.unit as u

In [3]:
template = peng.snapshot_from_pdb(
    "/Users/jan-hendrikprinz/Studium/git/openpathsampling/examples/resources/AD_initial_frame.pdb")
topology = peng.to_openmm_topology(template)

# Generated using OpenMM Script Builder
# http://builder.openmm.org

forcefield = mm.app.ForceField(
    'amber96.xml',  # solute FF
    'tip3p.xml'     # solvent FF
)

# OpenMM System
system = forcefield.createSystem(
    topology,
    nonbondedMethod=mm.app.PME,
    nonbondedCutoff=1.0*u.nanometers,
    constraints=mm.app.HBonds,
    ewaldErrorTolerance=0.0005
)

# OpenMM Integrator
integrator = mm.LangevinIntegrator(
    300 * u.kelvin,
    1.0 / u.picoseconds,
    2.0 * u.femtoseconds
)
integrator.setConstraintTolerance(0.00001)

# Engine options
options = {
    'n_steps_per_frame': 2,
    'n_frames_max': 1000
}

engine = peng.Engine(
    template.topology,
    system,
    integrator,
    options=options
)

In [5]:
t.generate(engine, template, p.LengthEnsemble(100))

Trajectory[100]

In [4]:
t.generate.delay(engine, template, p.LengthEnsemble(100))

<AsyncResult: eb6dfe10-f4ea-4e5e-9101-8daf2379f822>

In [3]:
trajs = [generate.delay(engine, template, p.LengthEnsemble(x*10 + 5)) for x in range(20)]

In [12]:
for traj in trajs:
    print traj.result

Trajectory[5]
Trajectory[15]
Trajectory[25]
Trajectory[35]
Trajectory[45]
Trajectory[55]
Trajectory[65]
Trajectory[75]
Trajectory[85]
Trajectory[95]
Trajectory[105]
Trajectory[115]
Trajectory[125]
Trajectory[135]
Trajectory[145]
Trajectory[155]
Trajectory[165]
Trajectory[175]
Trajectory[185]
Trajectory[195]


In [37]:
ff = [traj.result for traj in trajs]

In [42]:
ff[0][2] == ff[1][1]

False

In [19]:
%%time
generate(engine, template, p.LengthEnsemble(1000))

CPU times: user 45.9 s, sys: 8.82 s, total: 54.7 s
Wall time: 25.7 s


Trajectory[1000]

In [20]:
%%time
trajs = [generate.delay(engine, template, p.LengthEnsemble(100)) for x in range(10)]

running = True
while running:
    running = False
    for traj in trajs:
        if not traj.successful():
            running = True

CPU times: user 7.58 s, sys: 1.24 s, total: 8.82 s
Wall time: 15.5 s
